<a href="https://colab.research.google.com/github/tomeliud/Explatory-analysis-using-r-/blob/master/Wining_solution_X_ray_TB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install nvidia-smi

In [ ]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



## Import Full-Size-Images

In [ ]:
!gdown https://drive.google.com/uc?id=1KdpV3M27kV-_QOQOrAentfzZ2tew8YS-&

Downloading...
From: https://drive.google.com/uc?id=1KdpV3M27kV-_QOQOrAentfzZ2tew8YS-
To: /content/images_fullsize.zip
4.38GB [00:57, 76.5MB/s]


In [ ]:
!unzip -q /content/images_fullsize.zip

In [ ]:
!pip install fastai --upgrade -q
!pip install -q efficientnet_pytorch

     |████████████████████████████████| 194kB 5.3MB/s 
     |████████████████████████████████| 61kB 5.4MB/s 


In [ ]:
!pip install --upgrade fastcore -q
!pip install --upgrade fastai -q

## Libraries-Env

In [ ]:
import random
import numpy as np
import gc
from fastai.vision.all import *
from fastai.vision import *
from pathlib import Path
import matplotlib.pyplot as plt
from efficientnet_pytorch import EfficientNet

In [ ]:
SEED_VAL  = 42
# Set the seed value all over the place to make this reproducible.
def seed_all(SEED):
  random.seed(SEED_VAL)
  np.random.seed(SEED_VAL)
  torch.manual_seed(SEED_VAL)
  torch.cuda.manual_seed_all(SEED_VAL)
  os.environ['PYTHONHASHSEED'] = str(SEED_VAL)
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False

In [ ]:
print('Make sure cuda is installed:', torch.cuda.is_available())
print('Make sure cudnn is enabled:', torch.backends.cudnn.enabled)

Make sure cuda is installed: False
Make sure cudnn is enabled: True


## Train-Submission

In [ ]:
def get_model(dls,pretrained=True, **kwargs):
    model = EfficientNet.from_pretrained(model_name)
    model._fc = nn.Linear(model._fc.in_features, dls.c)
    return model

In [ ]:
#params
model_name = 'efficientnet-b0'

def X_Rays_FASTAI_First_Place_Solution() :
  seed_all(SEED_VAL)
  df = pd.read_csv("/content/Train.csv")
  df2 = pd.read_csv("/content/Test.csv")
  dls = ImageDataLoaders.from_df(df,
                               path='/content', 
                               label_col="LABEL",
                               item_tfms=RandomResizedCrop(512, min_scale=0.5),
                               batch_tfms=aug_transforms(mult=1.0, do_flip=True, 
                                                        flip_vert=True, max_rotate=10., 
                                                        max_zoom=1.1,max_warp=0.2, 
                                                        p_affine=0.75, p_lighting=0, 
                                                        xtra_tfms=None),
                               valid_pct=0.03,
                               bs=8,
                               val_bs=8)
  # define - Learner
  learn = Learner(dls=dls,model= get_model(dls), metrics=[accuracy])
  print("\n `\n Start Training ....")
  learn.fit_one_cycle(20,1e-3)
  print("\n `\n Unfreeze & Again Training ....")
  learn.unfreeze()
  learn.fit_one_cycle(5,1e-4)
  print("\n `\n Start PREDICTING ....")
  test_image_ids = [img.split(".")[0] for img in os.listdir("/content/test") ]
  test_images = [f"/content/test/{img}.png" for img in test_image_ids]
  preds = learn.get_preds(dl=dls.test_dl(test_images, shuffle=False, drop_last=False))
  imglabels = list(df.columns[1:])
  ls = preds[0]
  lss = list(ls[0])
  vocab = list(dls[0].dataset.vocab)
  submission = pd.DataFrame()
  submission["ID"] = test_image_ids
  for i in range(2):
    submission[vocab[i]] = ls[:, i]
  submission=submission.drop([0], axis=1)
  return submission

In [ ]:
submission = X_Rays_FASTAI_First_Place_Solution()
submission.to_csv("effb0.csv", index=False)

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b0-355c32eb.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b0-355c32eb.pth



Loaded pretrained weights for efficientnet-b0

 `
 Start Training ....


epoch,train_loss,valid_loss,accuracy,time
0,0.623487,0.744320,0.428571,20:30
1,0.544751,0.746327,0.285714,20:29
2,0.522088,1.104756,0.476190,20:33
3,0.522610,0.547474,0.714286,20:39
4,0.526137,3.116190,0.428571,20:32
5,0.543114,0.462427,0.809524,20:30
6,0.505429,0.313034,0.904762,20:31
7,0.490713,0.727957,0.666667,20:36
8,0.449061,0.581640,0.714286,20:33
9,0.423753,1.301287,0.571429,20:35



 `
 Unfreeze & Again Training ....


epoch,train_loss,valid_loss,accuracy,time
0,0.187958,0.159605,0.904762,20:35
1,0.213172,0.155257,0.904762,20:42
2,0.234122,0.150432,0.904762,20:47
3,0.197624,0.126174,0.904762,20:48
4,0.209643,0.115080,0.904762,20:52



 `
 Start PREDICTING ....
